In [1]:
import sys
import re
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

% matplotlib inline

# 1. Write a simulator as a positive control

In [2]:
#specify parameters to be used in simulation

#specify number of transcripts to initialize random probability vector 
K = 10

#specify random probability vector for the transcript abundance using Dirichlet 
tau = np.random.dirichlet(np.ones(K))

#specify transcript identities
transcripts = ['Arc1', 'Arc2', 'Arc3', 'Arc4', 'Arc5', 'Arc6', 'Arc7', 'Arc8', 'Arc9', 'Arc10']

#specify segment identities
Segs = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

#specify the number of read counts we want to simulate
N = 1000000

#specify the isoform lengths in terms of number of segments (in same order as transcript isoforms)
#change if you want!
L = [4, 2, 3, 4, 4, 3, 2, 2, 3, 3]

#change transcript abundances to nt abundances
v = np.divide(np.multiply(tau, L), np.sum(np.multiply(tau, L)))

In [3]:
def simulate_read_counts(N, transcripts, v, L):
    
    #specify seed to keep consistent random number generation
    np.random.seed(10)

    #specify empty reads matrix based on number of segments
    reads = np.zeros(len(Segs))

    #simulate N read counts from the reads
    for i in range(N):
        #choose a transcript
        t = np.random.choice(len(transcripts), p=v)
        #get length of chosen transcript
        l = L[t] 
        #choose segment 
        s = np.random.randint(t,t+l)
        #account for circularity
        if s > 9:
            s = s - 10
        reads[s] = reads[s] + 1 
    
    return reads

In [4]:
reads = simulate_read_counts(N, transcripts, v, L)

In [5]:
print("read counts:", reads, "sum of read counts:", np.sum(reads), "nt abundances:", v, "probabilities:", tau)

('read counts:', array([101026., 181175., 158928.,  95096.,  36009.,  51471., 116963.,
       160113.,  61739.,  37480.]), 'sum of read counts:', 1000000.0, 'nt abundances:', array([0.25559672, 0.16202955, 0.04288536, 0.06879471, 0.0167199 ,
       0.09123402, 0.12994345, 0.12014053, 0.00509722, 0.10755853]), 'probabilities:', array([0.17103957, 0.21685305, 0.03826388, 0.04603587, 0.01118858,
       0.08140233, 0.17391046, 0.16079067, 0.00454792, 0.09596766]))


# 2. Calculate the log likelihood

In [6]:
def llh_calculation(reads, transcripts, v, L):
    
    """
    Calculates the loglikelihood of the observed read counts given the reads, transcript isoforms, nucleotide 
    abundances, and the length of each transcript in the number of segements. 
    
    Parameters
    ----------
    
    reads: numpy array
        read counts of each segment corresponding to a transcript isoform.
    
    transcripts: list
        transcript isoform identities
    
    v: numpy array
        nucleotide abundances of each transcript isoform 
    
    L: list
        number of locus segments that make up each transcript isoform
        
    
    Returns
    -------
    
    llh: numpy float (negative)
        loglikelihood of the observed read counts given the parameters specified above
    
    """
    
    #initialize loglikelihood
    llh = 0
    #need to marginalize out the probability of transcript and probability of segment to get P(read|v)
    for ri in range(len(reads)):  
        #initialize p at zero 
        pi = 0
        for ti in range(len(transcripts)):
            #iterate through segments of the chosen transcript 
            for si in range(ti, ti + L[ti]):
                #circularity
                if si > 9:
                    si = si - 10
                #check to see that the read is actually corresponding to the 'true' segment 
                if ri == si:
                    pi = pi + v[ti]/np.float(L[ti])
        #account for the number of read counts per read in llh and log pi        
        llh = llh + np.log(pi)*reads[ri]  
    return llh

In [7]:
llh_me = llh_calculation(reads, transcripts, v, L)

In [8]:
llh_me

-2168867.9609611654

In [9]:
#reminder of the simulated reads
reads

array([101026., 181175., 158928.,  95096.,  36009.,  51471., 116963.,
       160113.,  61739.,  37480.])

In [10]:
#remind ourselves of Lestrade's code now
T = len(transcripts)

S = len(Segs)

r = reads

segusage = np.zeros(S).astype(int)
for i in range(T):
    for j in range(i,i+L[i]): 
        segusage[j%S] += 1


# Naive analysis:
c  = np.zeros(T)
for i in range(T):
    for k in range(i,i+L[i]):
        c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                              # and assign 1/usage count to each transcript
                                                              # that contains segment j.
Z       = np.sum(c)
est_nu  = np.divide(c, Z)       # nucleotide abundance

In [11]:
#Lestrade's estimated nuceleotide abundance for each transcript isoform
est_nu

array([0.17874167, 0.11336767, 0.09667767, 0.09009942, 0.098429  ,
       0.086426  , 0.069269  , 0.07089775, 0.08328483, 0.112807  ])

In [12]:
#true nucleotide abundances for each transcript isoform from the positive control set in part 1
v

array([0.25559672, 0.16202955, 0.04288536, 0.06879471, 0.0167199 ,
       0.09123402, 0.12994345, 0.12014053, 0.00509722, 0.10755853])

In [13]:
#use Lestrade's nucleotide abundance and see how it affects the llh
llh_lestrade = llh_calculation(reads, transcripts, est_nu, L)

In [14]:
llh_lestrade

-2208986.6054417607

In [15]:
print(llh_me, llh_lestrade, llh_me - llh_lestrade)

(-2168867.9609611654, -2208986.6054417607, 40118.644480595365)


Lestrade's estimated nucleotide abundance of each transcript isoform is clearly contributing to his method's poor performance when applying the loglikelihood calculation. When Lestrade is calculating the nucleotide abundance he is basically looking at a segment and if that segment is shared among multiple transcripts, he is assuming that each of the transcripts equally contributed to the segment count, affecting the isoform abundances in the end and making his method perform poorly. In the next step, I will do a better job of estimating the isoform abundances. 

# 3. Estimate isoform abundances by EM

In [16]:
#read in Lestrade's data - thank you Sean Eddy and Lestrade :)

with open("w09-data.out.txt") as f:
    #   The first line is "The <n> transcripts of the sand mouse Arc locus"
    line  = f.readline()
    match = re.search(r'^The (\d+) transcripts', line)
    T     = int(match.group(1))

    # The next T lines are 
    #   <Arcn>  <true_tau> <L> <structure>
    # tau's may be present, or obscured ("xxxxx")
    tau       = np.zeros(T)
    L         = np.zeros(T).astype(int)
    tau_known = True   # until we see otherwise
    for i in range(T):
        fields    = f.readline().split()
        if fields[1] == "xxxxx":
            tau_known = False
        else:
            tau[i] = float(fields[1])
        L[i]      = int(fields[2])

    # after a blank line,
    # 'The <n> read sequences':
    line  = f.readline()
    line  = f.readline()
    match = re.search(r'The (\d+) read sequences', line)
    N     = int(match.group(1))

    # the next T lines are 
    #  <read a-j> <count>
    r = np.zeros(T).astype(int)
    for k in range(T):
        fields = f.readline().split()
        r[k]   = fields[1]


S = T    # S = R = T : there are T transcripts (Arc1..Arc10), S segments (A..J), R reads (a..j)
R = T
Slabel   = list(string.ascii_uppercase)[:S]               # ['A'..'J']        : the upper case labels for Arc locus segments 
Tlabel   = [ "Arc{}".format(d) for d in range(1,T+1) ]    # ['Arc1'..'Arc10'] : the labels for Arc transcript isoforms
Rlabel   = list(string.ascii_lowercase)[:T]               # ['a'..'j']        : lower case labels for reads

In [17]:
r

array([104299, 124988, 120019, 140129,  80599, 124526, 132530, 108319,
        34583,  30008])

In [18]:
def EM_calculation(T, r, n, L):
    
    """
    Expectation maximization to maximize the nucleotide abundance given the read count data parameters. 
    
    Parameters
    ----------
    T: integer
        number of transcripts
    
    r: numpy array
        read counts
    
    n: integer 
        number of times to iterate through EM calculations
    
    L: list
        number of segments for each transcript
    
    Returns
    -------
    
    my_nu: numpy array
        maximized array of nucleotide abundances for each read
    
    """
    my_nu = np.random.dirichlet(np.ones(K))

    for iterate in range(n):
        counts = np.zeros(T)
        #expectation step
        #need to marginalize out the probability of transcript and probability of segment to get P(read|v)
        for ri in range(len(r)):  
            #need probability matrix for each transcript 
            pi = np.zeros(T)
            for ti in range(T):
                #iterate through segments of the chosen transcript 
                for si in range(ti, ti + L[ti]):
                    #circularity
                    if si > 9:
                        si = si - 10
                    #check to see that the read is actually corresponding to the 'true' segment 
                    if ri == si:
                    #change our 'true' nucleotide abundance to the estimated nucleotide abundance 
                        pi[ti] = pi[ti] + my_nu[ti]/np.float(L[ti])
            #normalize probabilities
            pi = np.divide(pi, sum(pi))
            for tj in range(T):
                counts[tj] = counts[tj] + r[ri]*pi[tj]
        #maximization step
        my_nu = np.divide(counts, sum(counts))

    return my_nu

In [19]:
my_nu = EM_calculation(T, r, 10000, L)

In [20]:
my_new_nu = EM_calculation(T, r, 100000, L)

In [21]:
my_new_nu

array([0.297164 , 0.059373 , 0.0481245, 0.199186 , 0.059044 , 0.1799055,
       0.016008 , 0.051171 , 0.0269925, 0.0630315])

In [22]:
llh_my_nu = llh_calculation(r, transcripts, my_nu, L)

In [23]:
llh_my_new_nu = llh_calculation(r, transcripts, my_new_nu, L)

In [24]:
llh_est_nu = llh_calculation(r, transcripts, est_nu, L)

In [25]:
print(llh_my_nu, llh_my_new_nu, llh_est_nu)

(-2218066.831235943, -2218066.8312359434, -2262837.3343829094)


It appears that my llh has converged between the two different iterations of the EM calculation (10000 v 100000) so I can be confident that the optimal llh has been reached. As we can see, my true nucleotide abundance gives better results than Lestrade's estimated abundance 

In [26]:
r

array([104299, 124988, 120019, 140129,  80599, 124526, 132530, 108319,
        34583,  30008])

In [27]:
#need to use real read counts so re-define est_nu as est_nu_r
T = len(transcripts)

S = len(Segs)

r = r

segusage = np.zeros(S).astype(int)
for i in range(T):
    for j in range(i,i+L[i]): 
        segusage[j%S] += 1


# Naive analysis:
c  = np.zeros(T)
for i in range(T):
    for k in range(i,i+L[i]):
        c[i] += (1.0 / float(segusage[k%S])) * float(r[k%S])  # For each read k, assume read k-> segment j,
                                                              # and assign 1/usage count to each transcript
                                                              # that contains segment j.
Z       = np.sum(c)
est_nu_r  = np.divide(c, Z)       # nucleotide abundance

In [28]:
#Lestrade's est_nu_r
est_nu_r

array([0.163145  , 0.081669  , 0.11358233, 0.14821717, 0.12858725,
       0.10172092, 0.06021225, 0.04437125, 0.06706183, 0.091433  ])

In [29]:
#convert the nucleotide abundances back to transcript abundances
tau_my_new_nu = np.divide(np.divide(my_new_nu, L), np.sum(np.divide(my_new_nu, L)))
tau_est_nu_r = np.divide(np.divide(est_nu_r, L), np.sum(np.divide(est_nu_r, L)))

In [30]:
tau_my_new_nu

array([0.241093  , 0.09634017, 0.05205871, 0.16160218, 0.04790316,
       0.19461288, 0.025975  , 0.08303139, 0.02919915, 0.06818436])

In [31]:
tau_est_nu_r

array([0.12445724, 0.12460447, 0.1155303 , 0.11306935, 0.09809442,
       0.10346546, 0.09186736, 0.06769834, 0.06821196, 0.0930011 ])

It's clear that Lestrade is incorrect about his statement claiming that the transcripts are all expressed at about the same level - as they are not all expressed at the same level. Arc7 and Arc9 appear to be expressed at much lower expression levels than Lestrade reports. The most abundant two trasncripts are Arc1 and Arc6 representing over 40% of the population. Lestrade should have accounted for the fact that not all of the transcripts share segments equally, which I have done through calculating (and normalizing) the probability of the read coming from different transcripts for a given segment. 

In [32]:
nu_sim = EM_calculation(T, reads, 10000, L)

In [33]:
llh_nu_sim = llh_calculation(reads, transcripts, nu_sim, L)

In [34]:
llh_me_sim = llh_calculation(reads, transcripts, v, L)

In [35]:
llh_L_sim = llh_calculation(reads, transcripts, est_nu, L)

In [36]:
llh_nu_sim # EM

-2168862.373198392

In [37]:
llh_me_sim # Truth

-2168867.9609611654

In [38]:
llh_L_sim # Lestrade

-2208986.6054417607

Comparing to the simulated data, it seems  that my method works better than Lestrades using simulated nucleotide abundances, as the llh is almost exactly the same as the llh with the true nucleotide abundances of the simulated data. It seems that my llh with simulated abundances may even be better than the llh with the true abundances, probably because of stochasticity (noise in true tau) in the simulation. And if Lestrade's method were actually better or just as good, we would see similar llh values here in the simulated case, but we don't. So, I am confident that my method is better than his. 